# System of non-linear equations - Coupled chemical equilibria
Author: Björn Dahlgren, Applied Physcial Chemistry, KTH Royal Insitiute of Technology

In this example we will study the equilibria between aqueous cupric ions and ammonia.
We will use [ChemPy](https://github.com/bjodah/chempy) which is a Python package collecting functions and classes useful for chemistry related problems. We will also make use of SymPy for manipulating and inspecting the formulae encountered.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import defaultdict
from chempy import atomic_number
from chempy.chemistry import Species, Equilibrium
from chempy.equilibria import EqSystem, NumSysLin, NumSysLog, NumSysSquare
from chempy.units import rescale, default_units as u
from IPython.display import Latex, display
import matplotlib.pyplot as plt
%matplotlib inline
def show(s):  # convenience function
    display(Latex('$'+s+'$'))
import sympy; import chempy; print('SymPy: %s, ChemPy: %s' % (sympy.__version__, chempy.__version__))

In [ ]:
#from IPython.core.display import display, HTML
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
if False:
    display(HTML('''<style>
.prompt{%(hide)s}
.prompt_container{%(hide)s}
.prompt.input_prompt{%(hide)s}
.out_prompt_overlay.prompt{%(hide)s}
</style>''' % dict(hide="width: 0px; min-width: 0px; visibility: collapse")))

Let's define our species with names and composition, ChemPy can parse chemical formulae:

In [ ]:
NH3_complexes = ['CuNH3+2', 'Cu(NH3)2+2', 'Cu(NH3)3+2', 'Cu(NH3)4+2', 'Cu(NH3)5+2']
OH_complexes = ['Cu2(OH)2+2', 'Cu(OH)3-', 'Cu(OH)4-2']
substances = [
    Species.from_formula(n) for n in ['H+', 'OH-', 'NH4+', 'NH3', 'H2O', 'Cu+2'] + 
        NH3_complexes + OH_complexes + ['Cu(OH)2(s)']
] #, CuOHp, CuOH2,
ionic_radii = {
    'NH4+': 2.5*u.angstrom,
    'H+': 9*u.angstrom,
    'OH-': 3.5*u.angstrom,
    'Cu+2': 6*u.angstrom,
} # Kielland (1937)
for substance in substances:
    if substance.charge != 0:
        substance.data['ionic_radius'] = rescale(ionic_radii['Cu+2' if substance.name.startswith('Cu') else substance.name], u.m)
substance_names = [s.name for s in substances]

Let's see how the species are pretty-printed:

In [ ]:
show(', '.join([s.latex_name for s in substances]))

Let's define some initial concentrations. We will consider different amount of added ammonia in 10 mM solutions of $Cu^{2+}$:

In [ ]:
init_conc = defaultdict(float, {'H+': 1e-7, 'OH-': 1e-7, 'NH4+': 0,
                                'NH3': 1.0, 'Cu+2': 1e-2, 'H2O': 55.5})

Now, let us define the equilibria, data are from course material at Applied Physcial Chemistry, KTH Royal Insitiute of Technology.

In [ ]:
H2O_c = init_conc['H2O']
w_autop = Equilibrium({'H2O': 1}, {'H+': 1, 'OH-': 1}, 10**-14/H2O_c)
NH4p_pr = Equilibrium({'NH4+': 1}, {'H+': 1, 'NH3': 1}, 10**-9.26)
CuOH2_s = Equilibrium({'Cu(OH)2(s)': 1}, {'Cu+2': 1, 'OH-': 2}, 10**-18.8)
CuOH_B3 = Equilibrium({'Cu(OH)2(s)': 1, 'OH-': 1}, {'Cu(OH)3-': 1}, 10**-3.6)
CuOH_B4 = Equilibrium({'Cu(OH)2(s)': 1, 'OH-': 2}, {'Cu(OH)4-2': 1}, 10**-2.7)
Cu2OH2 = Equilibrium({'Cu+2': 2, 'H2O': 2}, {'Cu2(OH)2+2': 1, 'H+': 2}, 10**-10.6 / H2O_c**2)
CuNH3_B1 = Equilibrium({'CuNH3+2': 1}, {'Cu+2': 1, 'NH3': 1}, 10**-4.3)
CuNH3_B2 = Equilibrium({'Cu(NH3)2+2': 1}, {'Cu+2': 1, 'NH3': 2}, 10**-7.9)
CuNH3_B3 = Equilibrium({'Cu(NH3)3+2': 1}, {'Cu+2': 1, 'NH3': 3}, 10**-10.8)
CuNH3_B4 = Equilibrium({'Cu(NH3)4+2': 1}, {'Cu+2': 1, 'NH3': 4}, 10**-13.0)
CuNH3_B5 = Equilibrium({'Cu(NH3)5+2': 1}, {'Cu+2': 1, 'NH3': 5}, 10**-12.4)
equilibria = w_autop, NH4p_pr, CuNH3_B1, CuNH3_B2, CuNH3_B3, CuNH3_B4, CuNH3_B5, Cu2OH2, CuOH2_s, CuOH_B3, CuOH_B4

### Export to C++...

In [ ]:
print("// std::vector<std::string> names = {%s}" % ", ".join(f'"{n}"' for n in substance_names))
print("std::vector<std::vector<eqs::IdxCoeff>> stoich;")
for i, eq in enumerate(equilibria):
    print('stoich.push_back({%s}); /* %s */' % (", ".join(
        ["{%d, %d}" % (substance_names.index(k), -v) for k, v in eq.reac.items()]+
        ["{%d, %d}" % (substance_names.index(k), +v) for k, v in eq.prod.items()]
    ), eq.string()))
print("std::array<double, %d> eq_consts {%s};" % (len(equilibria), ", ".join([str(eq.param) for eq in equilibria])))
nprecip = 1
print("return eqs::EqSys(%d, %d, stoich);" % (len(substances)-nprecip, nprecip))

### Human readable

Let's see if we can print ``equilibria`` in a human-readable form:

In [ ]:
import sympy as sp
net = sp.Matrix([eq.net_stoich(substance_names[:-3]) for eq in equilibria[:-3]])
ns = net.nullspace()
def print_mat(mat, empty_zeros=False):
    print('   ' + ' '.join("{:>11s}".format(e) for e in substance_names[:len(mat[0])]))
    print('\n'.join(f"{1+ri:>2d}:" + ' '.join(map(lambda e: " "*11 if empty_zeros and e==0 else "{:11.2f}".format(float(e)), row)) for ri, row in enumerate(mat)))

print_mat(ns, empty_zeros=True)

In [ ]:
print_mat(net.tolist(), empty_zeros=True)

In [ ]:
print('\n'.join(e.string() for e in equilibria))

In [ ]:
compo = sp.Matrix([[s.composition.get(c, 0) for c in [0, 1, 7, 8, atomic_number('Cu')]] for s in substances[:-3]]).T
#print(compo.shape)
print_mat(compo.tolist())

In [ ]:
compo_rref, compo_piv = compo.rref()
print(compo_piv)
print_mat(compo_rref.tolist())

In [ ]:
print_mat(net.rref()[0].tolist())  # not what we are looking for

In [ ]:
Q, R = net.QRdecomposition()
Q, R

In [ ]:
R.rref()[0]

In [ ]:
net.rref()[0]

In [ ]:
print_mat(net.rref()[0].tolist())

In [ ]:
def to_cxx(mat, flat=False):
    if flat:
        return ",\n".join([", ".join("{: 3g}".format(float(e)) for e in row) for row in mat.tolist()])+';\n'
    else:
        return str(mat.tolist()).replace('[', '{').replace(']', '}')

In [ ]:
to_cxx(net.rref()[0].applyfunc(float))

### Invariants

In [ ]:
def nullspace(mat):
    rows = mat.nullspace()
    for row in rows:
        assert len(row) == len(rows[0])
    return sp.Matrix(len(rows), len(rows[0]), lambda i, j: rows[i][j])
print_mat(nullspace(net).tolist())

In [ ]:
print_mat(nullspace(net).rref()[0].tolist())

In [ ]:
L, U, lu_piv = compo.LUdecomposition()
L, U, lu_piv

In [ ]:
sp.Matrix(U.tolist()[:-1]).rref()

In [ ]:
R = sp.Matrix(compo_rref.tolist()[:-1])
N = sp.Matrix([list(n) for n in ns])
R, N

In [ ]:
N.rref()

In [ ]:
import functools
N_gcd = functools.reduce(sp.gcd, N)
N/N_gcd

In [ ]:
net

In [ ]:
N@net.T

In [ ]:
import numpy as np
s=np.array([[float(e.strip()) for e in row.strip().split()] for row in """\
   0.49550276185810738   -0.36616683957796192  -0.082300703508099163  -0.064667961140072741                                            
   0.15672389166339035    0.53956433987380703   0.083352064813416907    0.15185588423140142                                            
   0.52041884415681294   -0.26082719466049581   0.076406010493367593   -0.12979582474815846                                            
  0.024916082298705577    0.10533964491746588    0.15870671400146669  -0.065127863608085743                                            
   0.65222665352149778    0.17339750029584497  0.0010513613053178064   0.087187923091328612                                            
  -0.10896152795961236   -0.46780479991621987   -0.10445292134310068    0.28838316393791613                                            
 -0.084045445660906826   -0.36246515499875392   0.054253792658366072    0.22325530032983038                                            
 -0.059129363362201225   -0.25712551008128803    0.21296050665983274    0.15812743672174467                                            
 -0.034213281063495631   -0.15178586516382209    0.37166722066129931   0.092999573113658973                                            
-0.0092971987647900121  -0.046446220246356335    0.53037393466276606   0.027871709505573147                                            
  0.015618883533915445    0.05889342467110948     0.6890806486642328  -0.037256154102512484                                            
  0.095524727407555896     0.1435190799151741  -0.042201713059367388    0.88047809633863505 """.split('\n')])
s

In [ ]:
to_cxx(s)

In [ ]:
s.T

In [ ]:
from scipy.linalg import lu
p,l,u=lu(s.T)

In [ ]:
np.set_printoptions(linewidth=200, formatter={'float': lambda x: "{:5.2f}".format(x) if x !=0 else "     "})
u

In [ ]:
def _piv(row):
    abs_avg = np.sum(np.abs(row))/row.size
    thresh = 1e-8 * abs_avg
    for i in range(len(row)):
        if abs(row[i]) > thresh:
            return i
    raise ValueError("failed to find pivot")
    
def reduced(mtx):
    rows = []
    for i, row in enumerate(mtx):
        piv = _piv(row)
        for j, prev in enumerate(rows):
            alpha = prev[piv]/row[piv]
            rows[j] = prev - alpha * row
        row /= row[piv]
        rows.append(row)
    if isinstance(mtx, np.ndarray):
        result = np.empty_like(mtx)
        for i, row in enumerate(rows):
            result[i, :] = row
    elif isinstance(mtx, sp.MatrixBase):
        result = sp.Matrix(len(rows), len(rows[0]), lambda i, j: rows[i][j])
    else:
        result = rows
    return result

#reduced(u)
print_mat(reduced(u).tolist())

In [ ]:
to_cxx(reduced(u))

In [ ]:
def pos(mtx):
    mtx = mtx.copy()
    for col_idx in range(mtx.shape[1]):
        col = mtx[:, col_idx]
        i_max = np.argmax(col)
        mx = col[i_max]
        if mx <= 0:
            raise ValueError("assumption failed")
        for i in range(mtx.shape[0]):
            coeff = mtx[i, col_idx]
            if coeff < 0:
                alpha = -coeff/mx
                mtx[i, :] += alpha*mtx[i_max, :]
    return mtx
pos_red = pos(reduced(u))
print_mat(pos_red.tolist())

In [ ]:
to_cxx(pos(reduced(u)))

In [ ]:
R@net.T

In [ ]:
u@net.T

In [ ]:
reduced(u)@net.T

In [ ]:
s.T@net.T

In [ ]:
d = np.array([float(e) for e in "1, 0, 0, 0, 0.31629267024806201, 1, 0, 0, 1.0502844468621562, 0.18882495495922746, 1, 0, 0.050284446862156079, 0.18882495495922716, 0.99999999999999956, 0, 1.3162926702480622, 0.99999999999999978, 6.8319789749995289e-16, 0, 0, 0, 0, 1, 0.050284446862156051, 0.18882495495922746, 1.0000000000000002, 1.0000000000000002, 0.10056889372431217, 0.37764990991845471, 1.9999999999999998, 1.0000000000000002, 0.15085334058646824, 0.56647486487768173, 2.9999999999999982, 1, 0.2011377874486244, 0.75529981983690875, 3.9999999999999982, 1, 0.25142223431078037, 0.94412477479613599, 4.9999999999999982, 1.0000000000000004, 0.63258534049612392, 2, 1.4432899320127035e-15, 2.0000000000000004".split(',')]).reshape((4, -1))
d

In [ ]:
def symbolic_sum(mat):
    for row in mat if isinstance(mat, list) else mat.tolist():
        sum_ = sum([sp.Symbol(f"[{s}]")*c for s, c in zip(substance_names, row)])
        if sum_ != 0:
            print(sum_)
symbolic_sum(compo_rref)

In [ ]:
#for vec in ns:
#    print(sum([sp.Symbol(f"[{s}]")*c for s, c in zip(substance_names, vec)]))
symbolic_sum(ns)

## Reduced non-linearity

In [ ]:
import sys; sys.version, sys.executable, np.__version__

In [ ]:
#from scipy.optimize import fmin
def fmin(f, x0):
    xmin = None
    vmin = None
    for x in range(-5,5):
        if x == 0:
            continue
        if xmin is None:
            xmin = x
            vmin = f(x)
            continue
        v = f(x)
        if v < vmin:
            xmin = x
            vmin = v
            #print(x, v)
    return xmin, vmin

def calc_eq_component_presence(A, subst_compo, *, threshold):
    result = []
    for i in range(A.rows):
        result.append([])
        for j in range(subst_compo.shape[0]):
            present = False
            for k in range(A.cols):
                if A[i, k] == 0:
                    continue
                if abs(subst_compo[j, k]) > threshold:
                    present = True
                    break
            result[i].append(present)
    return np.array(result, dtype=bool)

def min_mag(A, *, eq_compo_pres, aug=None):
    A = A.copy()
    rows = []
    for i in range(A.rows):
        min_abs_nonzero_ele = min([abs(x) for x in A[i, :] if x != 0])
        if (min_abs_nonzero_ele < 1 and min_abs_nonzero_ele > 0.1) or min_abs_nonzero_ele >= 2:
            print(i)
            A[i, :] /= min_abs_nonzero_ele
            if aug is not None:
                aug[i, :] /= min_abs_nonzero_ele
                
    for i in range(A.rows-1, -1, -1):
        def wsum(lst):
            return sum([e**2 for e in lst])
        little_more_than_one = 1 + 1e-10
        

        jmin = -1
        vmin = wsum(A[i, :])
        
        for j in range(0, A.rows):
            if i == j:
                continue
            
            candidate_row_contains_extra_compo = False
            for k in range(eq_compo_pres.shape[1]):
                if eq_compo_pres[j, k] and not eq_compo_pres[i, k]:
                    candidate_row_contains_extra_compo = True
                    break
            if candidate_row_contains_extra_compo:
                # we don't want to introduce an additional component just to reduce non-linearity
                # if this for some reason would be desired (guarantted that all components are present)
                # then `eq_compo_pres` can be set to all True
                continue
            
            def f(alpha):
                return wsum(A[i, :] + alpha*A[j, :])

            alpha, val = fmin(f, 1)
            #print(alpha, end=", ")
            if val*little_more_than_one < vmin:
                vmin = val
                jmin = j
                amin = alpha
        if jmin != -1:
            print(i, jmin, amin)
            #rows.append(A[i, :] + float(amin)*A[jmin, :])
            A[i, :] += amin*A[jmin, :]
            if aug is not None:
                aug[i, :] += amin*aug[jmin, :]
    return A

In [ ]:
net#.rref()

In [ ]:
sum(abs(e) for e in net.rref()[0])

In [ ]:
net

In [ ]:
eq_compo_pres = calc_eq_component_presence(net, pos_red, threshold=1e-10)
print_mat(eq_compo_pres.tolist(), empty_zeros=True)

In [ ]:
aug = sp.eye(8)
mmag = min_mag(net, aug=aug, eq_compo_pres=eq_compo_pres)
print_mat(mmag.tolist(), empty_zeros=True)

In [ ]:
#mmag2 = mmag.subs({sp.Float(1.0): sp.S.One, sp.Float(0.5): sp.S.One/2, sp.Float(2.0): 2*sp.S.One, sp.Float(3.0): 3*sp.S.One})
#mmag2, mmag2.rank()
mmag2 = mmag

In [ ]:
mmag3 = min_mag(mmag2, aug=aug, eq_compo_pres=eq_compo_pres)
mmag3

In [ ]:
sum(e != 0 for e in mmag3)

In [ ]:
mmag4 = min_mag(mmag3, aug=aug, eq_compo_pres=eq_compo_pres)
mmag4

In [ ]:
sum(e != 0 for e in mmag4)

In [ ]:
print_mat(mmag4.tolist(), empty_zeros=True)

In [ ]:
mmag5 = min_mag(mmag4, aug=aug, eq_compo_pres=eq_compo_pres)
mmag5

In [ ]:
sum(e != 0 for e in mmag5)

In [ ]:
aug

In [ ]:
ref = sp.Matrix(8,20, [0]*8*20)
ref[:, :12] = mmag5
ref[:, 12:] = aug
print(to_cxx(ref, flat=True))
ref

In [ ]:
#mmag5.rref()  # <--- worse

In [ ]:
aug

In [ ]:
import numpy as np
lbk = [np.log2(eq.param) for eq in equilibria[:8]]
lbk2 = aug@sp.Matrix(8, 1, lbk)
print(to_cxx(lbk2, flat=True).replace('\n', ' '))

In [ ]:
print(lbk2)

In [ ]:
#print(().applyfunc(
#    lambda x:
#    #x/np.log2(10)
#    exp2(x)
#)

### Lower bounds

In [ ]:
ic12 = [init_conc[k] for k in substance_names[:-3]]
print(ic12)

from functools import reduce
from operator import add
ns_mat = sp.Matrix(4, 12, reduce(add, map(list, ns)))
print(ns_mat)

ns_mat @ sp.Matrix(12, 1, ic12)  # note that this is *not* useful

In [ ]:
nsr = ns_mat.rref()[0]
#print_mat(nsr.tolist())
iv0 = nsr @ sp.Matrix(12, 1, ic12)  # this is what we are looking for
iv0
ub = [min(iv0[ri]/nsr[ri, ci] for ri in range(4) if abs(nsr[ri, ci]) > 1e-10) for ci in range(12)]
print(ub)  # nope

In [ ]:
print(iv0)

In [ ]:
pnsr = pos(nsr)
iv0 = pnsr @ sp.Matrix(12, 1, ic12)  # this is what we are looking for
iv0
ub = [min(iv0[ri]/pnsr[ri, ci] for ri in range(4) if abs(pnsr[ri, ci]) > 1e-10) for ci in range(12)]
print(ub)  # yup

In [ ]:
print_mat(pnsr.tolist())

In [ ]:
m, n, t = sp.symbols("m n t")
ft = (m+n*sp.exp(-t))/(1+sp.exp(-t))
ft

In [ ]:
dfdt = ft.diff(t)
dfdt

In [ ]:
dfdt.factor()

In [ ]:
from abc import abstractmethod
import warnings
import copy

class Result:
    def __init__(self, x, *, niter, mxiter):
        self.x = x
        self.niter = niter
        self.mxiter = mxiter
        
    @property
    def success(self):
        return self.niter < self.mxiter


class SingleEqProblemBase:
    def __init__(self, row, *, ci, lb, ub, K):
        self.ns = len(lb)
        assert self.ns == len(ub)
        assert self.ns == len(row)
        assert row[ci] < -1e-10
        self.row = np.asarray(row, dtype=np.float64)
        self.lb = np.asarray(lb, dtype=np.float64)
        self.ub = np.asarray(ub, dtype=np.float64)
        assert np.all(self.lb < self.ub)
        assert np.all(self.lb >= 0)
        #self.ci = ci
        assert 0 < K
        assert K < float('inf')
        self.K = K
        self.xmin = -min(ll/self.row[li] for li, ll in enumerate(self.lb) if self.row[li] > 1e-10)
        self.xmax = min(uu/-self.row[ui] for ui, uu in enumerate(self.ub) if self.row[ui] < -1e-10)
        
    
class SingleEqProblem(SingleEqProblemBase):
    
    def __init__(self, row, *, ci, lb, ub, K, log=False):
        super().__init__(row, ci=ci, lb=lb, ub=ub, K=K)
        self.log = log

    def _lo_hi(self):
        span = self.xmax-self.xmin
        return self.xmin-0*span, self.xmax+0*span

    def x(self, y, be=np):
        assert False
    
    def xprime(self, y, be=np):
        assert False

    def y(self, x, be=np):
        assert False

    def consu(self, x, be=np):
        if self.log:
            y = x; del x
            result = 0.0
            for i, c in enumerate(self.row):
                if c < -1e-10:
                    m = self.ub[i]
                    arg = m + c*self.x(y, be=be)
                    if isinstance(arg, float) and arg < 1e-150:
                        arg = 1e-150
                    result += -c*be.log(arg)
        else:
            result = 1.0
            for i, c in enumerate(self.row):
                if c < -1e-10:
                    result *= (self.ub[i] + c*x)**-c
        return result

    def produ(self, x, be=np):
        if self.log:
            y = x; del x
            result = 0.0
            for i, c in enumerate(self.row):
                if c > 1e-10:
                    l = self.lb[i]
                    arg = l+c*self.x(y, be=be)
                    if isinstance(arg, float) and arg < 1e-150:
                        arg = 1e-150
                    result += c*be.log(arg)
        else:
            result = 1.0
            for i, c in enumerate(self.row):
                if c > 1e-10:
                    result *= (self.lb[i] + c*x)**c
        return result

    def f(self, x, be=np):
        if self.log:
            return be.log(self.K) + self.consu(x, be=be) - self.produ(x, be=be)
        else:
            return self.K*self.consu(x) - self.produ(x)
    
    def df(self, x, be=np):
        if self.log:
            return self.df_log(x, be=be)
        else:
            return self.df_lin(x)
    
    def df_lin(self, x):
        terms = []
        consumed = self.consu(x)
        produced = self.produ(x)
        for i, c in enumerate(self.row):
            if c < -1e-10:
                denom = (self.ub[i] + c*x)
                if isinstance(denom, float):
                    assert denom >= 0
                
                if denom == 0:
                    rdenom = 1.0
                else:
                    rdenom = 1.0/denom
                terms.append(self.K*consumed*(-c)*c*rdenom)
            elif c > 1e-10:
                denom = (self.lb[i] + c*x)
                if isinstance(denom, float):
                    assert denom >= 0
                
                if denom == 0:
                    rdenom = 1.0
                else:
                    rdenom = 1.0/denom
                terms.append(-produced*c*c*rdenom)
        return sum(terms)

    def df_log(self, y, be=np):
        terms = []
        for i, c in enumerate(self.row):
            if c < -1e-10:
                m = self.ub[i]
                denom = m + c*self.x(y, be=be)
                #if isinstance(denom, float) and denom < 1e-150:
                #    denom = 1e-150
                terms.append(-c*c*self.xprime(y, be=be)/denom)
                if isinstance(terms[-1], float):
                    assert not np.isnan(terms[-1])
            elif c > 1e-10:
                l = self.lb[i]
                denom = l + c*self.x(y, be=be)
                #if isinstance(denom, float) and denom < 1e-150:
                #    denom = 1e-150
                terms.append(-c*c*self.xprime(y, be=be)/denom)
                if isinstance(terms[-1], float):
                    assert not np.isnan(terms[-1])
        return sum(terms)
    
    def linesearch(self, x, dx, lmbda, mxiter=15):
        fold = self.f(x+lmbda*dx)
        shrink = 0.7
        for i in range(mxiter):
            fnew = self.f(x + shrink*lmbda*dx)
            if 1.0001*abs(fnew) > abs(fold):
                return lmbda
            else:
                lmbda *= shrink
        warnings.warn("reached mxiter in linesearch")
        return lmbda
    
    def solve(self, guess=None, mxiter=40):
        print(f"{self.xmax=} {self.xmin=}")
        if guess is None:
            x = self.xmin + (self.xmax - self.xmin)/3.1234567899999 # guess 
            if self.log:
                x = self.y(x)
        else:
            x = copy.copy(guess)
        
        if not self.log:
            assert self.xmin <= x and x <= self.xmax
        
        fx = self.f(x)
        fx_old = float('inf')
        dfx = self.df(x)
        dx = -fx/dfx
        dx_old = float('inf')
        print(f"f(x)={fx} f(x+dx)={self.f(x+dx)} f(x-dx)={self.f(x-dx)} {self.K=}")
        i = 1
        lmbda=1.0
        while True:
            if self.log:
                if abs(dx) < 1e-7 and abs(fx) < 1e-6:
                    break
                if abs(dx) < 1e-5 and abs(fx) < 1e-4:
                    if abs(dx/dx_old - 1) < 0.01 and abs(fx/fx_old) > 0.95:
                        break
            else:
                if abs(dx)/abs(x) < 1e-10 and abs(fx)/(1 if self.log else self.K) < 1e-10:
                    break
            lmbda = self.linesearch(x, dx, lmbda*1.05)
            print(f"{i=:2d}  {x=:<19.16f}  {dx=:<19.16f}  {abs(fx)=:<19.16f}  {dfx=:<19.16f}  {lmbda=:<19.16f}")
            x += dx
            if self.log:
                if x < self.y(self.xmin):
                    x = self.y(self.xmin)
                elif x > self.y(self.xmax):
                    x = self.y(self.xmax)
            else:
                if x < self.xmin:
                    x = self.xmin
                elif x > self.xmax:
                    x = self.xmax
            fx_old = fx
            fx = self.f(x)
            if self.log:
                dfx = self.df(x)
            else:
                dfx = self.df(1e-200 + (1-6e-16)*x)
            dx_old = dx
            dx = -fx/dfx
            assert not (np.isinf(dx) or np.isnan(dx))
            i += 1
            if i >= mxiter:
                break
        print(f"{i=:2d}  {x=:<19.16f}  {dx=:<19.16f}  {abs(fx)=:<19.16f}  {dfx=:<19.16f}  {lmbda=:<19.16f}")
        return Result(x=x, niter=i, mxiter=mxiter)
    
class SingleEqProblemTanh(SingleEqProblem):
    def x(self, y, be=np):
        #assert self.log
        if isinstance(y, float) and y > 150:
            y = 150
        elif isinstance(y, float) and y < -150:
            y = -150
        ey = be.exp(-y)
        lo, hi = self._lo_hi()
        return (hi + lo*ey)/(1+ey)
    
    def xprime(self, y, be=np):
        ey = be.exp(y)
        lo, hi = self._lo_hi()
        return (hi-lo)*ey/(ey+1)**2
        
    def y(self, x, be=np):
        assert self.log
        lo, hi = self._lo_hi()
        a = x-lo
        b = hi - x
        if isinstance(a, float) and a < 1e-150:
            a = 1e-150
        if isinstance(b, float) and b < 1e-150:
            b = 1e-150
        return be.log(a) - be.log(b)
    
class SingleEqProblemAsinh(SingleEqProblem):
    def x(self, y, be=np):
        #assert self.log
        if isinstance(y, float) and y > 1e150:
            y = 1e150
        elif isinstance(y, float) and y < -1e150:
            y = -1e150
        return be.arcsinh(y)
    
    def xprime(self, y, be=np):
        if isinstance(y, float) and y > 1e150:
            y = 1e150
        elif isinstance(y, float) and y < -1e150:
            y = -1e150
        return 1/be.sqrt(y*y+1)
        
    def y(self, x, be=np):
        if isinstance(x, float) and x < -346:
            x = -346
        if isinstance(x, float) and x > 346:
            x = 346
        return be.sinh(x)

In [ ]:
a = np.array([-1, 2, 3, 4])
np.cumprod([a+2 for a in a])

In [ ]:
from scipy.special import binom

def prod(factors):
    return reduce(mul, factors)

class SingleEqProblemPoly(SingleEqProblemBase):
    def _poly(self):
        # P: K*Prod((u_i - c_i*x)**c_i, ∀ i ∈ consump.) - Prod((c_i*x)**c_i, ∀ i ∈ produ.)
        ma_consu = self.row < -1e-10
        ma_produ = self.row > +1e-10
        nconsu = np.sum(ma_consu)
        nprodu = np.sum(ma_produ)
        consu_idxs = [i for i, p in enumerate(ma_consu) if p]; assert len(consu_idxs) == nconsu
        produ_idxs = [i for i, p in enumerate(ma_produ) if p]; assert len(produ_idxs) == nprodu
        consu_coeffs = self.row[ma_consu]; assert len(consu_coeffs) == nconsu
        produ_coeffs = self.row[ma_produ]; assert len(produ_coeffs) == nprodu
        tot_abspwr_consu = np.prod(-consu_coeffs); assert tot_abspwr_consu >= 0
        tot_pwr_produ = np.prod(produ_coeffs); assert tot_pwr_produ >= 0
        max_totpwr = round(max(tot_abspwr_consu, tot_pwr_produ))
        pcoeffs = np.zeros(max_totpwr+1)
        #n_binom_terms = 
        
        ccc = np.cumsum(-consu_coeffs)
        print(f"{nconsu=}")
        if nconsu:
            for i in range(2**round(tot_abspwr_consu)):
                term = 1
                order = 0
                for j in range(round(tot_abspwr_consu)):
                    for k in range(nconsu):
                        if ccc[k] > j:
                            break
                    else:
                        assert False
                    if (i >> j) & 1:
                        term *= self.ub[k]
                        print(f": {i=} {j=} {k=}")
                    else:
                        term *= self.xmax*consu_coeffs[k]
                        order += 1
                        print(f"| {i=} {j=} {k=}")
                    #c = round(-consu_coeffs[j])
                    #k = i % (c+1)
                    #i = i // (c+1)
                    #c_choose_k = binom(c, k)
                    #term *= c_choose_k*self.ub[j]**(c-k)*(-c)**(k)
                    #order += k
                pcoeffs[order] += term
        pcoeffs *= self.K
        
        if nprodu:
            term = 1
            ppow = 0
            for c in map(round, produ_coeffs):
                term *= (self.xmax*c)**c
                ppow += c
            pcoeffs[ppow] -= term
        return np.polynomial.Polynomial(pcoeffs
                                        /np.max(pcoeffs))

    def solve(self):
        p = self._poly()
        for r in p.roots():
            if r.imag == 0 and 0 <= r.real <= self.xmax:
                return r.real
        raise ValueError("Failed to find root")

In [ ]:
sepp3 = SingleEqProblemPoly([-3, 2], ci=0, lb=[1e-3]*2, ub=[5, 5*2/3], K=7)
#p = sepp3._poly()
#p.roots()

#spoly = 7*(5-3*x)**3-(2*x)**2
#print(spoly.expand())
#sols=sympy.solveset(spoly)
#[s.evalf() for s in sols]

sepp3.solve()

In [ ]:
sepp3._poly()

In [ ]:
sepp = SingleEqProblemPoly([-2, -2, 1, 2], ci=1, lb=[1e-3]*4, ub=[2, 2, 1, 2], K=3)

In [ ]:
sepp._poly()

In [ ]:
x = sp.Symbol('x', real=True)
reac, prod = 1, 1
for i, c in enumerate(sepp.row):
    if c > 0:
        prod = prod * ((c*sepp.xmax*x)**int(c))
    elif c == 0:
        continue
    else:
        reac = reac * (sepp.ub[i] + c*sepp.xmax*x)**-int(c)
sympol = (sepp.K*reac - prod).expand()
mxcoeff = max([t/x**i for i, t in enumerate(sympol.as_ordered_terms()[::-1])])
sympol1 = sympol/mxcoeff
sympol1

In [ ]:
sepp.solve()

In [ ]:
sep = SingleEqProblem([-2, -2, 1, 2], ci=1, lb=[1e-3]*4, ub=[2, 2, 1, 2], K=3)
sep.solve()

In [ ]:
#sep_asinh = SingleEqProblemAsinh([-2, -2, 1, 2], ci=1, lb=[1e-3]*4, ub=[2, 2, 1, 2], K=3)
#sep_asinh.log = True
#r = sep_asinh.solve()
#print(sep_asinh.x(r.x))

In [ ]:
#from sym import Backend
#be = Backend()
#be.expm1

In [ ]:
#x = sp.Symbol('x')
#fs = sep_asinh.f(x, be=be)
#fs

In [ ]:
#def massage(expr):
#    return (expr.factor()/192).simplify().subs({1.0: 1})
#fps = sep_asinh.df(x, be=be)
#massage(fps)
#fps

In [ ]:
#fsd = fs.diff(x)
#massage(fsd)
#fsd

In [ ]:
#(fps - fsd).rewrite(sp.exp).simplify()

<pre>
fig, all_axes = plt.subplots(2, 2, figsize=(14,8))
for log, axes in zip([False, True], all_axes):
    sep_asinh.log = log
    #if log:
    #    xplt = np.linspace(0, 1, 500)
    #else:
    xplt = 10.0**np.linspace(-5, 0, 500)
    fx = [sep_asinh.f(x) for x in xplt]
    dfx = np.diff(fx)
    dx = np.diff(xplt)
    axes[0].plot(xplt, fx, label="f(x)")
    xplt2 = xplt[:-1]+0.5*dx
    dfdx_num = dfx/dx
    axes[0].plot(xplt2, dfdx_num, ls='--', label="f'(x) finite diff")
    if log:
        axes[0].plot(xplt2, [sep_asinh.df(x2) for x2 in xplt2], label="f'(x)")
        #for ax in axes:
        #    ax.set_yscale('symlog')
    else:
        axes[0].plot(xplt, [sep_asinh.df(1e-200 + (1-3e-16)*x) for x in xplt], label="f'(x)")
    axes[0].legend()
    axes[1].plot(xplt2, [sep_asinh.df(x2) for x2 in xplt2] - dfdx_num)
</pre>

In [ ]:
class LowerBoundsCalcBase:
    def __init__(self, net, Kv):
        self.net = net.tolist()
        self.Kv = Kv
        for row in self.net[1:]:
            assert len(row) == self.ns
        assert len(Kv) == self.ne

    @property
    def ne(self):
        return len(self.net)
    
    @property
    def ns(self):
        return len(self.net[0])

In [ ]:
print('Primary data')
print('============')
print('\n'.join(eq.string(with_param=True) for eq in equilibria[:-3]))
print("")
print('ic:'+' '.join(map("{0:11.4g}".format, ic12)))
print("\n\n")
print("Derived data")
print("============\n")
print_mat(net.tolist(), empty_zeros=True)
print("-"*146)
print_mat(pnsr.tolist(), empty_zeros=True)
print("-"*146)
print('ub:'+' '.join(map("{0:11.4g}".format, ub)))

In [ ]:
def walk_graph(ri, ci, *, eqs, level):
    to_walk = []  # breath first
    for i in range(12):
        if i == ci:
            continue
        if net[ri, i] == 0:
            continue
        is_reactant = eqs[ri][1]
        opposite_sign = is_reactant ^ (net[ri, i] < 0)
        for j in range(8):
            if net[j, i] == 0:
                continue
            if j in eqs and eqs[j][0] <= level:
                continue
            eqs[j] = (level, opposite_sign ^ (net[i, j] < 0))
            to_walk.append((j, i))
    for j, i in to_walk:
        walk_graph(j, i, eqs=eqs, level=level+1)

from collections import defaultdict

all_contribs = []
for ci in range(12):
    contribs = []
    level = 0
    eqs = {}
    for ri in range(8):
        if net[ri, ci] == 0:
            continue
        is_reactant = net[ri, ci] < 0
        eqs[ri] = (0, is_reactant)
        walk_graph(ri, ci, eqs=eqs, level=level+1)
    all_contribs.append(eqs)
all_contribs

In [ ]:
print_mat(mmag5.tolist(), empty_zeros=True)

In [ ]:
class LowerBoundsCalcInvar(LowerBoundsCalcBase):    
    def calc(self):
        for si in range(self.ns):
            #Kvals_subproblem = [float('inf')] * self.ne
            sub_problem_rows = []
            for ei, row in enumerate(self.net):
                if row[si] < -1e-10:
                    # consumed
                    sub_problem.append(row)
                if row[si] > 1e-10:
                    #produced
                    pass
                

    def __init__(self, *, net, Kv, pos_linvar, ics):
        super().__init__(net=net, Kv=Kv); del net; del Kv
        self.pos_linvar = pos_linvar.tolist()
        assert(len(self.pos_linvar) == self.ns - self.ne)
        for row in self.pos_linvar:
            assert len(row) == self.ns
        self.iv0 = pos_linvar @ ics
        assert(len(self.invar0) == len(self.pos_linvar))
        for iv in self.invar0:
            assert iv >= 0
                


In [ ]:
class LowerBoundCalcIter(LowerBoundsCalcBase):
    def __init__(self, *, net, Kv, ub, guess=None, niter=None
                #,lb=None
                ):
        super().__init__(net=net, Kv=Kv); del net; del Kv
        #if lb is None:
        self.lb = np.zeros(self.ns)
        #else:
        #    self.lb = lb
        assert len(self.lb) == self.ns
        self.ub = np.asarray(ub)
        assert np.all(self.ub > 0)
        assert np.all(self.lb < self.ub)
        self.niter = niter or []
        self.guess = guess or [None]*self.ns
        self.guess_is_log = False

    def iterate(self, log=False):
        consumptions = [[] for _ in range(self.ns)]
        niter = 0
        for ci in range(self.ns):
            for ei, row in enumerate(self.net):
                c = row[ci]
                if c < -1e-10:  # reactant
                    print(ci, ei)
                    seqp = SingleEqProblemAsinh(row, ci=ci, lb=self.lb, ub=self.ub, K=self.Kv[ei], log=log)
                    g = self.guess[ei]
                    if g is not None and (self.guess_is_log ^ log):
                        g = (seqp.y if log else seqp.x)(g)
                    res = seqp.solve(guess=g)
                    self.guess[ei] = res.x
                    niter += res.niter
                    if log:
                        x = seqp.x(res.x)
                    else:
                        x = res.x
                    consumptions[ci].append(-c*x)
                if c > 1e-10:  # product
                    print(ci, ei)
                    seqp = SingleEqProblemAsinh([-re for re in row], ci=ci, lb=self.lb, ub=self.ub, K=1/self.Kv[ei], log=log)
                    g = self.guess[ei]
                    if g is not None and (self.guess_is_log ^ log):
                        g = (seqp.y if log else seqp.x)(g)
                    res = seqp.solve(guess=g)
                    self.guess[ei] = res.x
                    niter += res.niter
                    if log:
                        x = seqp.x(res.x)
                    else:
                        x = res.x
                    consumptions[ci].append(c*x)
        self.guess_is_log = log
        print(consumptions)
        result = [uu-sum(terms) for uu, terms in zip(ub, consumptions)]
        self.lb = np.clip(result, 0, self.ub)
        self.niter.append(niter)
        return result

In [ ]:
Kvals = [eq.param for eq in equilibria[:8]]
lbc = LowerBoundCalcIter(net=net, ub=ub, Kv=Kvals)
lbc.iterate(log=True)

In [ ]:
print(lbc.niter, '\n', dict(zip(substance_names[:-3], lbc.lb)))

In [ ]:
lb_old = lbc.lb
lbc.iterate(log=False)

In [ ]:
print(lbc.niter, '\n', dict(zip(substance_names[:-3], lbc.lb)), lbc.lb - lb_old)

In [ ]:
lb_old = lbc.lb
lbc.iterate(log=True)

In [ ]:
print(lbc.niter, '\n', dict(zip(substance_names[:-3], lbc.lb)), lbc.lb - lb_old)

In [ ]:
lb_old = lbc.lb
lbc.iterate(log=True)

In [ ]:
print(lbc.niter, '\n', dict(zip(substance_names[:-3], lbc.lb)), lbc.lb - lb_old)

In [ ]:
Kvals = [eq.param for eq in equilibria[:8]]
def calc_lb(lb, ub, K):
    lb0 = lb.copy()
    for ri, row in enumerate(net.tolist()):
        for ci, coeff in enumerate(row):
            if coeff == 0:
                continue
            #g = guess.copy()
            Q = 1
            if coeff < -1e-10:  # reactant
                for ii, cc in enumerate(row):
                    if -1e-10 < cc < 1e-10:
                        continue
                    if ii == ci:
                        continue
                    if cc < 0:
                        Q *= lb0[ii]**cc
                    else:
                        Q *= ub[ii]**cc
                lb[ci] = min(lb[ci], (Q/Kvals[ri])**(1.0/-coeff))
            else:  # product
                if coeff < 0:
                    coeff = 0
                for ii, cc in enumerate(row):
                    if -1e-10 < cc < 1e-10:
                        continue
                    if ii == ci:
                        continue
                    if cc < 0:
                        Q *= ub[ii]**cc
                    else:
                        Q *= lb0[ii]**cc
                lb[ci] = min(lb[ci], (Kvals[ri]/Q)**(1.0/coeff))
    return lb

In [ ]:
lb0 = ub.copy()
lb1 = calc_lb(lb0, ub, Kvals)
print(lb1)

In [ ]:
lb2 = calc_lb(lb1, ub, Kvals)
print(lb2)

### NumEqSys

In [ ]:
show(', '.join([s.latex_name for s in substances]))
show('~')
from math import log10
for i, eq in enumerate(equilibria):
    ltx = eq.latex(dict(zip(substance_names, substances)))
    show(str(i+1) + ":~~" + ltx + '~'*(80-len(ltx)) + 'lgK = {0:12.5g}'.format(log10(eq.param)))  # latex table would be better...

To keep our numerical treatment as simple as possible we will try to avoid representing
$Cu(OH)_2(s)$ explicitly (which is present in the three last equilibria). This is becuase the
system of equations change when precipitation sets in.

However, we do want to keep the two last equilibria, therefore we rewrite
those using the dissolution equilibria them only using dissolved species:

In [ ]:
new_eqs = CuOH2_s - CuOH_B3, CuOH2_s - CuOH_B4
[str(_) for _ in new_eqs]

Now it's time to exclude the precipitate species and replace the last three equilibria with our two new ones:

In [ ]:
#skip_subs, skip_eq = (4, 4) # (0, 0), (1, 1), (3, 3), (4, 4), (11, 9)
#skip_subs, skip_eq = (1, 3)
skip_subs, skip_eq = (3, 3)
simpl_subs = substances[:-skip_subs]
simpl_eq = equilibria[:-skip_eq] # + new_eqs
simpl_c0 = {k.name: init_conc[k.name] for k in simpl_subs}

From the law of mass action we can from the equilbria and from the preservation of mass and charge formulate a non-linear system of equations:

In [ ]:
import sympy as sp
import numpy as np
sp.init_printing()
eqsys = EqSystem(simpl_eq, simpl_subs)
x, i, Ks = sp.symarray('x', eqsys.ns), sp.symarray('i', eqsys.ns), sp.symarray('K', eqsys.nr)
correct_debye_huckel_extended = False
params = np.concatenate((i, [K*(
    eq.Kcorr_activity(
        molalities=dict(zip(eqsys.substances, x)),
        substances=eqsys.substances,
        kw=dict(
            eps_r=80.1,
            rho=998*u.kg/u.m3,
            T=298.15*u.K,
            units=u,
            backend=sp
        )).replace(
    lambda e: isinstance(e, sp.ImmutableDenseNDimArray) and e.shape == (),
    lambda e: e.args[0].args[0]
) if correct_debye_huckel_extended else 1) for K, eq in zip(Ks, eqsys.rxns)]))
params

<font color='gray'>*double click to view code for generating a graph*</font>
<!--
from IPython.display import Image
from chempy.util.graph import rsys2graph
from itertools import chain
from chempy import ReactionSystem
rsys8_12 = ReactionSystem(chain(*[eq.as_reactions(kb=1) for eq in equilibria[:8]]), substances[:12])
rsys2graph(rsys8_12, 'ammonical_cupric.png', save='.', include_inactive=False)
Image('ammonical_cupric.png')
-->

In [ ]:
numsys_lin = NumSysLin(eqsys, backend=sp)
f = numsys_lin.f(x, params)
f # print(f)

It turns out that the success of the numerical root finding process for above system of equations is terribly sensitive on the choice of the initial guess. We therefore reformulate the equations in terms of the logarithm of the concentrations:

In [ ]:
numsys_log = NumSysLog(eqsys, backend=sp)
f = numsys_log.f(x, params)
f

We can take a peek on the jacobian of this vector:

In [ ]:
def spy(j):
    print('\n'.join(' '.join([' #'[int(e != 0)] for e in row]) for row in j.tolist()))
j=sp.Matrix(1, len(f), lambda _, q: f[q]).jacobian(x)
spy(j)

The preservation equations of mass and charge actually contain a redundant equation, so currently our system is over-determined:

In [ ]:
len(f), eqsys.ns

We could cast the preservation equations into [reduced row echelon form](https://en.wikipedia.org/wiki/Row_echelon_form) (which would remove one equation), but for now we'll leave this be and rely on the Levenberg-Marquardt algorithm to solve our problem in a least-squares sense. (Levenberg-Marquardt uses [QR-decomposition](https://en.wikipedia.org/wiki/QR_decomposition) internally for which it is acceptable to have overdetermined systems).

Let's solve the equations for our inital concentrations:

In [ ]:
C, sol, sane = eqsys.root(simpl_c0, NumSys=NumSysLog)
assert sol['success'] and sane
C

Great, let's now vary the initial concentration of $NH_3$ and plot the equilibrium concentrations of our species:

In [ ]:
import numpy as np
plt.figure(figsize=(20,8))
NH3_varied = np.logspace(-4, 0, 200)
Cout_logC, extra, success = eqsys.roots(
    simpl_c0, NH3_varied, 'NH3', NumSys=NumSysLog, plot_kwargs={'latex_names': True})
all(success), sum([nfo['nfev'] for nfo in extra['info']]), sum([nfo['njev'] for nfo in extra['info']])
_ = plt.gca().set_ylim([1e-10, 60])

But the above diagram is only true if we are below the solubility limit of our neglected $\rm Cu(OH)_2(s)$.

Let's plot the solubility product in the same diagram:

In [ ]:
sol_prod = Cout_logC[:, eqsys.as_substance_index('Cu+2')]*Cout_logC[:, eqsys.as_substance_index('OH-')]**2
plt.figure(figsize=(20,6))
plt.loglog(NH3_varied, sol_prod, label='[$%s$][$%s$]$^2$' % (eqsys.substances['Cu+2'].latex_name,
                                                             eqsys.substances['OH-'].latex_name))
plt.loglog(NH3_varied, Cout_logC[:, eqsys.as_substance_index('H+')], ls=':',
           label='[$%s$]' % eqsys.substances['H+'].latex_name)
plt.loglog([NH3_varied[0], NH3_varied[-1]], [10**-18.8, 10**-18.8], 'k--', label='$K_{sp}(Cu(OH)_2(s))$')
plt.xlabel('[$NH_3$]')
_ = plt.legend()

We see that for a ammonia concentration exceeding ~500-600 mM we would not precipitate $Cu(OH)_2(s)$ even though our pH is quite high (almost 12).

We have solved the above system of equations for the *logarithm* of the concentrations. How big are our absolute and relative errors compared to the linear system? Let's plot them:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))
eqsys.plot_errors(Cout_logC, simpl_c0, NH3_varied, 'NH3', axes=axes)

Not bad. So the problem is essentially solved. But let's say that it is very important to know the exact position of the intersection for the solutbility limit. We can locate it using the secant method:

In [ ]:
from scipy.optimize import newton
convergence = []
def sol_lim(c_NH3):
    c0 = simpl_c0.copy()
    c0['NH3'] = c_NH3
    C, sol, sane = eqsys.root(c0, NumSys=NumSysLog)
    assert sol['success'] and sane
    prod = C[eqsys.as_substance_index('Cu+2')]*C[eqsys.as_substance_index('OH-')]**2
    discrepancy = prod/10**-18.8 - 1
    convergence.append(discrepancy)
    return discrepancy
Climit_NH3 = newton(sol_lim, 0.5)
convergence = np.array(convergence).reshape((len(convergence), 1))
print(convergence)
Climit_NH3

For fun, let's see what the equation system looks like if we canonicalize it by transforming the equations for equibliria and the equations for the preservation relations to their respective reduced row echelon form:

In [ ]:
len(x)

In [ ]:
numsys_log_rref = NumSysLog(eqsys, rref_equil=True, rref_preserv=True, backend=sp)
rf = numsys_log_rref.f(x, params)
rf

In [ ]:
len(eqsys.params()), len(Ks)

In [ ]:
len(rf)

In [ ]:
sp.Symbol('x') == sp.Symbol('x', nonnegative=True)

In [ ]:
fx = NumSysLin(eqsys, rref_equil=True, rref_preserv=True, backend=sp).f(x, params)
fx

<!-- sp.Matrix(1, len(fx), lambda _, q: fx[q]).jacobian(x) -->

In [ ]:
NumSysLin(eqsys, rref_equil=False, rref_preserv=True, backend=sp).f(x, params)

In [ ]:
from operator import mul
from aq_radiolysis.sym_util import safe_div

def exportable(*, log, rref_equil=True, max_small=False, wrap=False):
    if log:
        def fmt_i(i):
            return f"log(i{i})"
        objectives = NumSysLog(eqsys, rref_equil=rref_equil, rref_preserv=True, backend=sp).f(x, params)
    else:
        def fmt_i(i):
            return f"i{i}"
        objectives = NumSysLin(eqsys, rref_equil=rref_equil, rref_preserv=True, backend=sp).f(x, params)
        
        def no_denom(e):
            assert e.is_Add
            mult_by = []
            def remove_denom(d):
                mult_by.append(1/d)
                return d
            e2 = e.replace(lambda fact: fact.is_Pow and fact.exp < 0, remove_denom)
            return (e2*reduce(mul, [1] + mult_by)).expand().factor()
            
        objectives = list(map(no_denom, objectives))
        
    small1 = sp.Symbol("small1")
    small2 = sp.Symbol("small2")
    
    if max_small:
        modified = [e.replace(lambda e: e.is_Mul, safe_div) for e in objectives]
    else:
        modified = objectives

    if wrap:
        subs = dict(zip(x, map(lambda arg: sympy.Max(arg, small2), x)))
        modified = [e.xreplace(subs) for e in modified]
        
    def stringify(e):
        return str(e).replace('**', '^').replace('x_', 'x').replace("Max", "max").replace('i_', 'i')

    
    s = []
    s += [f"x{i}\t{m}\t{fmt_i(i)}\t0\t{s.name}" for i, (s, m) in enumerate(zip(simpl_subs, map(stringify, modified)))]
    s += ["", "small1\t1e-70", "small2\t1e-90"]
    s += [f"i{i}\t{init_conc[k.name] or small2}\t\t{s.name}" for i, (k, s) in enumerate(zip(simpl_subs, simpl_subs))]
    s += [f"{k}\t{v}" for k, v in zip(Ks, eqsys.params())]
    s += [""]
    
    if not log:
        s += [" ".join(f"comp1.x{i} 0" for i in range(len(simpl_subs)))]
    return "\n".join(s)

print(exportable(log=True))

In [ ]:
print(exportable(log=False, max_small=True, wrap=False, rref_equil=False))

In [ ]:
print(exportable(log=False, max_small=True))

In [ ]:
print(exportable(log=False))

So the Jacobian should be considerably more diagonally dominant now:

In [ ]:
j = sp.Matrix(1, len(rf), lambda _, q: rf[q]).jacobian(x)
spy(j)

In [ ]:
j.rows, j.cols

<!--
j = sp.Matrix(1, len(rf), lambda _, q: rf[q]).jacobian(x)
piv = []
for ir in range(j.rows):
    if j[ir, ir] != 0:
        piv.append(ir)
        continue
    for ir2 in range(ir+1, j.rows):
        if j[ir2, ir] != 0:
            piv.append(ir2)
            j.row_swap(ir, ir2)
            break
    else:
        raise ValueError("No pivot found")
j
-->

<!--
permute = list(range(len(piv)))
for i, j in enumerate(piv):
    tmp = permute[i]
    permute[i] = permute[j]
    permute[j] = tmp
permute
-->

And let's see if this system converges as well:

In [ ]:
plt.figure(figsize=(20,8))
out = eqsys.roots(simpl_c0, NH3_varied, 'NH3', 
                  rref_preserv=True, 
                  #rref_equil=True,
                  #NumSys=NumSysLog,
                  rref_equil=False,
                  NumSys=NumSysLin,
                  plot_kwargs={'latex_names': True})
_ = plt.gca().set_ylim([1e-10, 60])

Sinvce version 0.2.0 of chempy there is support for automatic reformulation of the system of equations when precipitation occurs:

In [ ]:
full_eqsys = EqSystem(equilibria[:-2] + new_eqs, substances)
full_numsys_log_rref = NumSysLog(full_eqsys, rref_equil=False, rref_preserv=False, precipitates=[False], backend=sp)
full_x, full_i, full_Ks = sp.symarray('x', full_eqsys.ns), sp.symarray('i', full_eqsys.ns), sp.symarray('K', full_eqsys.nr)
full_rf = full_numsys_log_rref.f(full_x, np.concatenate((full_i, full_Ks)))
full_rf

In [ ]:
def solve_and_plot_full(NumSys, plot_kwargs={'latex_names': True}, **kwargs):
    plt.figure(figsize=(18, 7))
    result = full_eqsys.roots(init_conc, NH3_varied, 'NH3', NumSys=NumSys, plot_kwargs=plot_kwargs or {}, **kwargs)
    
    try:
        cur_val = None
        for idx, condition in enumerate([nfo['intermediate_info'][0]['conditions'] for nfo in result[1]['info']]):
            any_precip = condition != (False,)*len(condition)
            if cur_val is None:
                if any_precip:
                    onset = idx
            elif cur_val == any_precip:
                pass
            else:
                if any_precip:
                    onset = idx
                else:
                    plt.axvspan(NH3_varied[onset], NH3_varied[idx], facecolor='gray', alpha=0.1)
                    onset = None
            cur_val = any_precip
        if onset is not None:
            plt.axvspan(NH3_varied[onset], NH3_varied[-1], facecolor='gray', alpha=0.1)
    except KeyError:
        pass
    return result

In [ ]:
xout_log, sols_log, sane_log = solve_and_plot_full(NumSysLog, solver='scipy', tol=1e-10, conditional_maxiter=30,
                                                   rref_equil=True, rref_preserv=True, method='lm')
plt.gca().set_ylim([1e-15, 60])
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')

We see that the numerical solution is not perfect (it could probably be improved by scaling the components individually). But the principle is clear: we can solve the solve non-linear system of equations using this method.

Let's see if we can gain some more insight here:

In [ ]:
def sum_species(x, species, substance_names, weights=None):
    accum = np.zeros(x.shape[0])
    if weights is None:
        weights = [1]*len(substance_names)
    for idx in map(substance_names.index, species):
        accum += x[:, idx]*weights[idx]
    return accum

def plot_groups(varied, x):
    substance_names = list(full_eqsys.substances.keys())
    weights = [s.composition.get(atomic_number('Cu'), ) for s in full_eqsys.substances.values()]
    amines = sum_species(x, NH3_complexes, substance_names, weights)
    hydroxides = sum_species(x, OH_complexes, substance_names, weights)
    free = sum_species(x, ['Cu+2'], substance_names, weights)
    precip = sum_species(x, ['Cu(OH)2(s)'], substance_names, weights)
    
    tot = amines + hydroxides + free + precip

    plt.figure(figsize=(13.7, 7))
    plt.plot(NH3_varied, tot, label='tot')
    plt.plot(NH3_varied, amines, label='amines')
    plt.plot(NH3_varied, hydroxides, label='hydroxides')
    plt.plot(NH3_varied, free, label='free')
    plt.plot(NH3_varied, precip, label='precip')
    plt.legend(loc='best')
    plt.gca().set_xscale('log')

plot_groups(NH3_varied, xout_log)

Without any precipitates (we force the system to not precipitate any solids, applicable for short timescales for example):

In [ ]:
xout_static, sols_static, sane_static = solve_and_plot_full(
    NumSysLog, solver='scipy', tol=1e-12, # , NumSysSquare
    neqsys_type='static_conditions',
    rref_equil=True, rref_preserv=True,
    precipitates=(False,), method='lm', plot_kwargs=None
)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.gca().set_ylim([1e-10, 60])

plot_groups(NH3_varied, xout_static)